In [167]:
import polars as pl
### Load the datasets
# g_patent.tsv
df_patent = pl.read_csv(
    "g_patent.tsv",
    separator="\t",
    columns=["patent_id", "patent_date"],
    schema_overrides={"patent_id": pl.Utf8, "patent_date": pl.Utf8},
    ignore_errors=True
).with_columns([
    pl.col("patent_date").str.strptime(pl.Date, "%Y-%m-%d", strict=False)
])

# g_cpc_current.tsv
df_cpc = pl.read_csv(
    "g_cpc_current.tsv",
    separator="\t",
    columns=["patent_id", "cpc_class"],
    schema_overrides={"patent_id": pl.Utf8, "cpc_class": pl.Utf8}
)

# g_assignee_disambiguated.tsv
df_assignee = pl.read_csv(
    "g_assignee_disambiguated.tsv",
    separator="\t",
    columns=["patent_id", "location_id"],
    schema_overrides={"patent_id": pl.Utf8, "location_id": pl.Utf8}
)

# g_location_disambiguated.tsv
df_location = pl.read_csv(
    "g_location_disambiguated.tsv",
    separator="\t",
    columns=["location_id", "disambig_country"],
    schema_overrides={"location_id": pl.Utf8, "disambig_country": pl.Utf8}
)
# g_cpc_title.tsv
df_cpc_title = pl.read_csv(
    "g_cpc_title.tsv",
    separator="\t",
    columns=["cpc_class", "cpc_class_title"],
    schema_overrides={"cpc_class": pl.Utf8, "cpc_class_title": pl.Utf8}
)


In [168]:
df_pa = df_cpc.join(df_assignee, on="patent_id", how="inner")


In [169]:
df_pal = df_pa.join(df_location, on="location_id", how="inner")

In [170]:
df_full = df_pal.join(df_patent, on="patent_id", how="inner")

In [171]:
df_full_unique = df_full.unique(subset=['patent_id'])

In [172]:
##Dataset with full patent information
print(df_full.height)
print(df_full.head(10))

54918973
shape: (10, 5)
┌───────────┬───────────┬─────────────────────────────────┬──────────────────┬─────────────┐
│ patent_id ┆ cpc_class ┆ location_id                     ┆ disambig_country ┆ patent_date │
│ ---       ┆ ---       ┆ ---                             ┆ ---              ┆ ---         │
│ str       ┆ str       ┆ str                             ┆ str              ┆ date        │
╞═══════════╪═══════════╪═════════════════════════════════╪══════════════════╪═════════════╡
│ 3950001   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9b5f-1234bd… ┆ CH               ┆ 1976-04-13  │
│ 3950001   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9b5f-1234bd… ┆ CH               ┆ 1976-04-13  │
│ 3950001   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9b5f-1234bd… ┆ CH               ┆ 1976-04-13  │
│ 3950002   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9b5f-1234bd… ┆ CH               ┆ 1976-04-13  │
│ 3950002   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9b5f-1234bd… ┆ CH               ┆ 1976-04-13  │
│ 3950002   ┆ A63       ┆ 8f95fbe8-16c8-11ed-9

In [173]:
import polars as pl
import numpy as np

# ------------------------------------------------------
# 1. Agregar columna de año
# ------------------------------------------------------
df_full = df_full.with_columns([
    pl.col("patent_date").dt.year().alias("year")
])

# ------------------------------------------------------
# 2. Crear grid completo país × clase × año
# ------------------------------------------------------
countries = df_full.select("disambig_country").unique()
classes = df_full.select("cpc_class").unique()
years = df_full.select("year").unique()
grid = countries.join(classes, how="cross").join(years, how="cross")

# ------------------------------------------------------
# 3. Calcular componentes RCA con nunique (por patente_id)
# ------------------------------------------------------
df_counts = df_full.group_by(["disambig_country", "cpc_class", "year"]).agg(
    pl.col("patent_id").n_unique().alias("patents_country_class_year")
)

df_total_country_year = df_full.group_by(["disambig_country", "year"]).agg(
    pl.col("patent_id").n_unique().alias("total_country_year")
)

df_total_class_year = df_full.group_by(["cpc_class", "year"]).agg(
    pl.col("patent_id").n_unique().alias("total_class_year")
)

df_total_year = df_full.group_by("year").agg(
    pl.col("patent_id").n_unique().alias("total_global_year")
)

# ------------------------------------------------------
# 4. Unir todo al grid y calcular RCA con protección contra división por cero
# ------------------------------------------------------
df_rca = (
    grid
    .join(df_counts, on=["disambig_country", "cpc_class", "year"], how="left")
    .join(df_total_country_year, on=["disambig_country", "year"], how="left")
    .join(df_total_class_year, on=["cpc_class", "year"], how="left")
    .join(df_total_year, on="year", how="left")
    .fill_null(0)
    .with_columns([
        pl.when(
            (pl.col("total_country_year") > 0) & 
            (pl.col("total_class_year") > 0) & 
            (pl.col("total_global_year") > 0)
        ).then(
            (pl.col("patents_country_class_year") / pl.col("total_country_year")) /
            (pl.col("total_class_year") / pl.col("total_global_year"))
        ).otherwise(0.0).alias("rca")
    ])
    .with_columns([
        (pl.col("rca") > 1).cast(pl.Int8).alias("rca_binary")
    ])
)

# ------------------------------------------------------
# 5. Exportar resultado final como pandas DataFrame
# ------------------------------------------------------
df_rca_full = df_rca.select(["disambig_country", "cpc_class", "rca_binary", "year"]).to_pandas()


In [174]:
print(df_rca_full.head(10))

  disambig_country cpc_class  rca_binary  year
0               EE       F16           0  1986
1               EE       F16           0  1980
2               EE       F16           0  1977
3               EE       F16           0  1983
4               EE       F16           0  1989
5               EE       F16           0  1992
6               EE       F16           0  2013
7               EE       F16           0  1998
8               EE       F16           0  1995
9               EE       F16           0  2007


In [175]:
de=df_rca_full[(df_rca_full['disambig_country'] == 'AND') & (df_rca_full['year'] == 1976)]

In [176]:
import numpy as np
import pandas as pd

def calc_discrete_proximity(mcp, ubiquity, asymmetric=False):
    """Calculate product proximity matrices - discrete

    Args:
        mcp: numpy ndarray with rows as locations and columns as products
        ubiquity: numpy array of shape=number of columns in "rca_or_mcp"
        asymmetric: Whether to generate asymmetric proximity matrix (True) or
            symmetric (False). *default* False.

    Returns:
        pandas df with proximity values for every product pair
    """
    ubiquity = mcp.sum(axis=0)
    ubiquity_safe = np.where(ubiquity == 0, 1, ubiquity)
    # Calculate discrete proximity
    phi = mcp.T @ mcp
    phi = phi / ubiquity_safe[np.newaxis, :]

    if asymmetric == False:
        # Symmetric proximity matrix
        phi = np.minimum(phi, phi.T)
    elif asymmetric == True:
        # Asymmetric proximity matrix
        phi = phi.T

    return phi


In [177]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


def calc_density(rca_or_mcp, proximity_mat, knn=None):
    """Calculate density, as defined by Hidalgo et. al. (2007)

    Args:
        rca_or_mcp: numpy array of RCA (if continuous product proximities are
            used), else Mcp
        proximity_mat: product proximity matrix
        knn: number of nearest neighbors to consider for density calculation (optional)

    Returns:
        numpy array of same shape as proximity_mat corresponding to density of
        each product
    """
    if knn is None:
        den = np.nansum(proximity_mat, axis=1)[np.newaxis, :]
        # density = rca_or_mcp @ (proximity_mat / den)
        den_safe = np.where(den == 0, 1, den)
        density = rca_or_mcp @ (proximity_mat.T / den_safe)
        
    else:
        # Convert proximity matrix to a distance matrix
        distance_mat = 1 - proximity_mat
        # Get proximity to k nearest neighbors
        nbrs = NearestNeighbors(n_neighbors=knn, metric="precomputed").fit(distance_mat)
        distance_knn, indices_knn = nbrs.kneighbors()
        # Get proximity
        proximity_knn = 1 - distance_knn
        # Calculate density
        # Get denominator
        den = np.nansum(proximity_knn, axis=1)
        density = []
        for i, row in enumerate(indices_knn):
            # Use row to subset rca_or_mcp
            rca_knn_p = rca_or_mcp[np.arange(rca_or_mcp.shape[0])[:, np.newaxis], row]
            # Get distance_knn for this row
            proximity_knn_row = proximity_knn[i]
            # Divide by den
            proximity_knn_row = proximity_knn_row / den[i]
            # Multiply each row of rca_knn_p by proximity_knn_row
            num_p = rca_knn_p * proximity_knn_row
            # Sum across columns
            density_p = np.nansum(rca_knn_p, axis=1)
            density.append(density_p)
        density = np.array(density).T
    return density

In [178]:
def add_relatedness_density(df):
    all_density = []

    for year in sorted(df['year'].unique()):
        # Filtrar por año
        df_year = df[df['year'] == year]
        
        # Crear matriz MCP (país × clase)
        mcp_df = df_year.pivot_table(index='disambig_country',
                                     columns='cpc_class',
                                     values='rca_binary',
                                     fill_value=0)
        mcp = mcp_df.values

        # Calcular relatedness
        phi = calc_discrete_proximity(mcp, None)

        # Calcular density
        density = calc_density(mcp, phi)

        # Volver a DataFrame
        df_density = pd.DataFrame(density, index=mcp_df.index, columns=mcp_df.columns).reset_index()

        df_density = df_density.melt(id_vars='disambig_country',
                                     var_name='cpc_class',
                                     value_name='relatedness_density')
        df_density['year'] = year
        all_density.append(df_density)

    # Unir resultados y agregar al df_rca_full
    df_density_all = pd.concat(all_density, ignore_index=True)
    df = df.merge(df_density_all, on=['disambig_country', 'cpc_class', 'year'], how='left')
    df['relatedness_density'] = df['relatedness_density'].fillna(0)

    return df


In [179]:
df_input = add_relatedness_density(df_rca_full)
print(df_input.head(10))
print(df_input.shape)

  disambig_country cpc_class  rca_binary  year  relatedness_density
0               EE       F16           0  1986             0.005312
1               EE       F16           0  1980             0.000000
2               EE       F16           0  1977             0.000000
3               EE       F16           0  1983             0.000000
4               EE       F16           0  1989             0.016109
5               EE       F16           0  1992             0.000000
6               EE       F16           0  2013             0.140315
7               EE       F16           0  1998             0.000000
8               EE       F16           0  1995             0.000000
9               EE       F16           0  2007             0.076048
(1157772, 5)


In [180]:
df_input = df_input.sort_values(by=["disambig_country", "cpc_class", "year"])

def compute_Mt(group):
    group = group.sort_values("year")
    group["M_t"] = (
        (group["rca_binary"] >= 1) & ##condicion actual
        (group["rca_binary"].shift(1) < 1) & # no la tenía en t−1
        (group["rca_binary"].shift(2) < 1) & # ni en t−2
        (group["rca_binary"].shift(3) < 1)  # ni en t−3
    ).astype(int)
    return group

df_input = df_input.groupby(["disambig_country", "cpc_class"], group_keys=False).apply(compute_Mt)


C:\Users\anyot\AppData\Local\Temp\ipykernel_26124\3742823992.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_input = df_input.groupby(["disambig_country", "cpc_class"], group_keys=False).apply(compute_Mt)


In [181]:
import pycountry

# Función para convertir de Alpha-2 a Alpha-3
def alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None

# Reemplazar directamente la columna 'country'
df_input["disambig_country"] = df_input["disambig_country"].apply(alpha2_to_alpha3)
print(df_input.head(30))


      disambig_country cpc_class  rca_binary  year  relatedness_density  M_t
36685              AND       A01           0  1976             0.000000    0
36654              AND       A01           0  1977             0.000000    0
36668              AND       A01           0  1978             0.000000    0
36689              AND       A01           0  1979             0.000000    0
36653              AND       A01           0  1980             0.000000    0
36669              AND       A01           0  1981             0.000000    0
36690              AND       A01           0  1982             0.000000    0
36655              AND       A01           0  1983             0.000000    0
36673              AND       A01           0  1984             0.000000    0
36686              AND       A01           0  1985             0.000000    0
36652              AND       A01           0  1986             0.000000    0
36674              AND       A01           0  1987             0.000000    0

In [182]:

df_gdp_raw = pd.read_csv("GDP.csv", skiprows=4, header=0)
df_gdp = df_gdp_raw.melt(
    id_vars=["Country Code"], 
    var_name="year", 
    value_name="gdp_per_capita"
)

# Convertir año (strings como '1960', '2020') a enteros, ignorando errores
df_gdp["year"] = pd.to_numeric(df_gdp["year"], errors="coerce")

# Eliminar filas donde year no sea un número
df_gdp = df_gdp.dropna(subset=["year"])

# Cambiar tipo a entero limpio
df_gdp["year"] = df_gdp["year"].astype(int)

# Renombrar
df_gdp.rename(columns={"Country Code": "country"}, inplace=True)
df_input.rename(columns={"disambig_country": "country"}, inplace=True)
print(df_gdp.head(10))

### GDP > x

    country  year gdp_per_capita
798     ABW  1960            NaN
799     AFE  1960     186.132432
800     AFG  1960            NaN
801     AFW  1960     121.938353
802     AGO  1960            NaN
803     ALB  1960            NaN
804     AND  1960            NaN
805     ARB  1960            NaN
806     ARE  1960            NaN
807     ARG  1960            NaN


In [183]:
df_input_ML = df_input.merge(df_gdp, on=["country", "year"], how="left")
green_classes = [
    "A01", "A23", "A43", "A47", "A61", "B01", "B03", "B09", "B22", "B29", "B30",
    "B62", "B63", "B65", "C02", "C03", "C04", "C05", "C08", "C09", "C10", "C12",
    "C21", "C22", "D01", "D21", "E01", "E02", "E03", "F01", "F02", "F16", "F17",
    "F23", "F27", "G01", "G08", "H01", "Y02"
]

ket_classes = [
    # Nanotechnology
    "Y01", "B82", 
    
    # Micro- and Nanoelectronics
    "H01", "H05", "F21", "Y01", "H03", 
    
    # Photonics
    "F21", "G02", "H01", "H02", "H05", 
    
    # Industrial Biotechnology
    "C02", "C07", "C12", "C08", "C09", "G01","A61",
    
    # Advanced Materials
    "B32", "C01", "C04", "C08", "C22", "D21", "H01","Y01",
    
    # Advanced Manufacturing Technologies
    "B03", "B06", "B07", "B23", "G01", "G05", "G06","G07", "G08",
    "A21", "A22", "A23", "A24", "A41", "A42", "A43", "B01", "B02", "B03", "B05",
    "B07", "B08", "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B30", "B31",
    "B41", "B42", "B44", "B65", "B67", "B68", "C13", "C14", "C23", "D01", "D02", "D03",
    "D04", "D05", "D06", "D21", "E01", "E02", "E21", "F04", "F16", "F26", "G01", "H05"
]

df_input_ML["is_green"] = df_input_ML["cpc_class"].isin(green_classes).astype(int)
print(df_input_ML.head(10).reset_index(drop=True),df_input_ML.shape)



  country cpc_class  rca_binary  year  relatedness_density  M_t  \
0     AND       A01           0  1976                  0.0    0   
1     AND       A01           0  1977                  0.0    0   
2     AND       A01           0  1978                  0.0    0   
3     AND       A01           0  1979                  0.0    0   
4     AND       A01           0  1980                  0.0    0   
5     AND       A01           0  1981                  0.0    0   
6     AND       A01           0  1982                  0.0    0   
7     AND       A01           0  1983                  0.0    0   
8     AND       A01           0  1984                  0.0    0   
9     AND       A01           0  1985                  0.0    0   

  gdp_per_capita  is_green  
0    7721.288586         1  
1    8167.922862         1  
2    9409.508513         1  
3   11996.407286         1  
4   12474.925292         1  
5   10465.260464         1  
6     9697.52202         1  
7     8082.09152         1  
8

In [184]:
# Contar valores nulos
na_count = df_input_ML['gdp_per_capita'].isna().sum()

# Contar valores igual a 0
zero_count = (df_input_ML['M_t'] == 0).sum()
Uno_count = (df_input_ML['M_t'] == 1).sum()
print(f"Unos: {zero_count}, Ceros: {Uno_count}")


Unos: 1140287, Ceros: 17485
